### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if True:
    import sys
    sys.path.append("/home/synopsis/git/CinemaNet-Training/")
    sys.path.append("/home/synopsis/git/CinemaNet-CLIP/")
    sys.path.append("/home/synopsis/git/YOLOX-Custom/")
    sys.path.append("/home/synopsis/git/YOLO-CinemaNet/")
    sys.path.append("/home/synopsis/git/icevision/")
    sys.path.append("/home/synopsis/git/labelling-workflows/")
    sys.path.append("/home/synopsis/git/amalgam/")
    sys.path.append("/home/synopsis/git/cinemanet-multitask-classification/")
    sys.path.append("/home/synopsis/git/Synopsis.py/")

from training.inference import InferenceModel
from upyog.all import *

import seaborn as sns
import open_clip
import torch

### Load Models

In [ ]:
open_clip.list_pretrained_tags_by_model("convnext_base_w")

In [ ]:
# Load either a pretrained model, or a trained checkpoint

# inf = InferenceModel.from_ckpt_path(
#     "training/logs/2023_05_09-12_54_19-model_convnext_base_w-lr_0.0001-b_256-j_8-p_amp_bf16/checkpoints/epoch_15.pt",
#     device=0, alpha=0.75, batch_size=128, path_embedding=None, experiment_name="oi"
# )

inf = InferenceModel("convnext_base_w", "laion_aesthetic_s13b_b82k", 0)

### Run Evaluations

In [ ]:
inf.eval_imagenet()

In [ ]:
inf.eval_cinemanet()

In [ ]:
self.eval_shotdeck_clip_datasets()